In [9]:
import os
import re
import rasterio
import xarray as xr
import numpy as np

# Define the folder containing the raster files
raster_folder = "E:\\Maps 2010\\National\\Rainfed\\Wheat2"

# Get a list of all TIFF files in the folder
raster_files = [os.path.join(raster_folder, file) for file in os.listdir(raster_folder) if file.endswith(".tif")]

# Create an empty list to store the datasets
datasets = []

# Regular expression pattern to extract the month from the filename
pattern = r".*_(\d+)\.tif"

# Iterate over each raster file
for raster_file in raster_files:
    # Extract the month from the filename using regex
    match = re.match(pattern, os.path.basename(raster_file))
    if match:
        month = int(match.group(1))
    else:
        print(f"Warning: Unable to extract month from filename: {raster_file}")
        continue

    # Open the raster file
    with rasterio.open(raster_file) as src:
        # Read raster data
        data = src.read(1, masked=True)
        transform = src.transform
        crs = src.crs.to_string()

        # Calculate the spatial coordinates
        height, width = data.shape
        rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing="ij")
        x, y = rasterio.transform.xy(transform, rows, cols, offset='center')

    # Flatten x, y coordinates for proper indexing
    x_coords = np.array(x).reshape(height, width)[0, :]
    y_coords = np.array(y).reshape(height, width)[:, 0]

    # Create a dataset using xarray
    ds = xr.Dataset(data_vars={'harvested_area': (('y', 'x'), data)},
                    coords={'x': x_coords, 'y': y_coords},
                    attrs={'crs': crs})

    # Add the month coordinate
    ds.coords['month'] = month

    # Append the dataset to the list
    datasets.append(ds)

# Concatenate datasets along the month dimension
merged_ds = xr.concat(datasets, dim='month')

# Write the merged dataset to a NetCDF file
merged_nc = "E:\\Products\\Monthly Harvested Area\\2005\\MIRCA-OS_Wheat_2_2005_rf.nc"
merged_ds.to_netcdf(merged_nc, format="NETCDF4_CLASSIC", engine="netcdf4")

print("Conversion and merging completed with spatial reference preserved.")
print(merged_nc)


Conversion and merging completed with spatial reference preserved.
E:\Products\Monthly Harvested Area\2005\MIRCA-OS_Wheat_2_2005_rf.nc


In [70]:
import rasterio
from rasterio.merge import merge
import numpy as np
import os

# Define the input and output directories
# input_dir = "E:\\Products\\Annual Harvested Area\\Annual Irrigated Harvested Area\\Wheat"
input_dir ="E:\\Maps 2010\\National\\Rainfed\\Sunflower"
output_dir = "E:\\Products\\Annual Harvested Area\\Absolute area\\2010"
# output_dir = "E:\\Products\\Annual Harvested Area\\Annual Rainfed Harvested Area\\Rice\\2010"
# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# List all .tif files in the input directory
raster_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]

# Open all raster datasets and store them in a list
datasets = [rasterio.open(file) for file in raster_files]

# Mosaic the datasets using the 'max' method
mosaic, out_transform = merge(datasets, method='max')

# Copy the metadata from the first dataset
out_meta = datasets[0].meta.copy()

# Update the metadata to reflect the number of layers
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_transform})

# Output file name
output_filename = "MIRCA-OS_Sunflower_2010_rf_1.0.tif"
output_path = os.path.join(output_dir, output_filename)

# Save the mosaic raster
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(mosaic)

# Close all datasets
for dataset in datasets:
    dataset.close()

print(f"Mosaic created and saved at {output_path}")


Mosaic created and saved at E:\Products\Annual Harvested Area\Absolute area\2010\MIRCA-OS_Sugar_cane_2010_rf_1.0.tif


In [ ]:
MIRCA-OS_Others_annual_2010_ir_1.0

In [25]:
import os
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import numpy as np

def sum_rasters(folder_path, output_file):
    rasters = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.tif'):  # Assuming TIFF format
            filepath = os.path.join(folder_path, filename)
            raster = rasterio.open(filepath)
            rasters.append(raster)

    # Check if all rasters have the same shape and CRS
    # You can also add resampling or reprojecting logic here if needed
    first_raster = rasters[0]
    for raster in rasters:
        if raster.shape != first_raster.shape or raster.crs != first_raster.crs:
            raise ValueError("Rasters have different shapes or CRS")

    # Read and sum the raster data
    sum_array = np.zeros(first_raster.shape, dtype=np.float32)
    for raster in rasters:
        sum_array += raster.read(1).astype(np.float32)  # Read the first band

    # Save the summed raster
    with rasterio.open(output_file, 'w', driver='GTiff', height=sum_array.shape[0], width=sum_array.shape[1], count=1, dtype=sum_array.dtype, crs=first_raster.crs, transform=first_raster.transform) as dst:
        dst.write(sum_array, 1)

# Usage
folder_path = "E:\\Products\\Annual Harvested Area\\Annual Irrigated Harvested Area\\otha\\2010"
output_file = "E:\\Products\\Annual Harvested Area\\Absolute area\\2010\\MIRCA-OS_Others_annual_2010_ir_1.0.tif"
sum_rasters(folder_path, output_file)


ValueError: Rasters have different shapes or CRS

In [10]:
import rasterio
import numpy as np
import os

def pad_raster(input_data, target_shape):
    """ Pad the input raster data with zeros to match the target shape. """
    padded_data = np.zeros(target_shape, dtype=input_data.dtype)
    padded_data[:input_data.shape[0], :input_data.shape[1]] = input_data
    return padded_data

# Define paths
input_dir = "E:\\Products\\Annual Harvested Area\\Absolute area\\2005"
output_dir = "E:\\Products\\Annual Harvested Area\\Precetange of grid cell area\\2005"
pixel_area_raster_file = "E:\\Products\\Annual Harvested Area\\Pixel area\\Cell_area.tif"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Open the pixel area raster
with rasterio.open(pixel_area_raster_file) as pixel_area_raster:
    pixel_area_data = pixel_area_raster.read(1)

# Process each raster file
for raster_file in os.listdir(input_dir):
    if raster_file.endswith('.tif'):
        input_file_path = os.path.join(input_dir, raster_file)

        # Open the input raster file
        with rasterio.open(input_file_path) as src:
            data = src.read(1)  # Read the first band

            # Pad raster if necessary
            if data.shape != pixel_area_data.shape:
                data = pad_raster(data, pixel_area_data.shape)

            # Initialize processed_data with zeros
            processed_data = np.zeros_like(data)

            # Find indices where pixel_area_data is greater than zero
            valid_indices = pixel_area_data > 0

            # Perform band math only where pixel_area_data is greater than zero
            processed_data[valid_indices] = (data[valid_indices] / pixel_area_data[valid_indices]) * 100

            # Prepare output file path
            output_file_path = os.path.join(output_dir, raster_file)

            # Copy metadata and update
            out_meta = src.meta.copy()
            out_meta.update({"height": processed_data.shape[0], "width": processed_data.shape[1]})

            # Write processed data to new file
            with rasterio.open(output_file_path, 'w', **out_meta) as dest:
                dest.write(processed_data, 1)

print("Processing complete. Processed rasters are saved in the output directory.")



C:\Users\endal\AppData\Local\Temp\ipykernel_5572\4280311420.py:43: RuntimeWarning: overflow encountered in multiply
  processed_data[valid_indices] = (data[valid_indices] / pixel_area_data[valid_indices]) * 100


Processing complete. Processed rasters are saved in the output directory.


In [96]:
import os
import shutil
# Path to your raster folder
source_folder_path = "D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\Asia\\All countries\\Maps 2010\\District\\Rainfed - Copy"


# Path to the destination folder where corrected files will be saved
destination_folder_path = "D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\Asia\\All countries\\Maps 2010\\District\\New folder"

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder_path):
    os.makedirs(destination_folder_path)

# Loop through each file in the source folder
for filename in os.listdir(source_folder_path):
    # Check if the file name ends with '.0' before the file extension
    if filename.endswith('.0.tif'):
        # Construct the full path to the current file in the source folder
        old_file = os.path.join(source_folder_path, filename)
        
        # Create the new file name by removing the '.0' before the extension
        new_filename = filename.replace('.0.tif', '.tif')
        new_file = os.path.join(destination_folder_path, new_filename)
        
        # Copy the file to the new location with the new file name
        shutil.copy2(old_file, new_file)
        print(f'Copied "{old_file}" to "{new_file}"')
    else:
        # If the file doesn't need renaming, just copy it as is
        shutil.copy2(os.path.join(source_folder_path, filename), os.path.join(destination_folder_path, filename))
        print(f'Copied "{filename}" without changes')



Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_1120000_rf_5.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_1120000_rf_5.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_1120000_rf_6.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_1120000_rf_6.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_1120000_rf_7.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_1120000_rf_7.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_1120000_rf_8.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_1120000_rf_8.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_4100002_rf_1.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_4100002_rf_1.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_4100002_rf_11.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_4100002_rf_11.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_4100002_rf_12.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_4100002_rf_12.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Bary_4100002_rf_2.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Bary_4100002_rf_2.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560003_rf_4.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560003_rf_4.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560003_rf_5.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560003_rf_5.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560003_rf_6.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560003_rf_6.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560003_rf_7.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560003_rf_7.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560033_rf_6.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560033_rf_6.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560033_rf_7.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560033_rf_7.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560033_rf_8.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560033_rf_8.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Casa_1560033_rf_9.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Casa_1560033_rf_9.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Cofe_1560001_rf_2.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Cofe_1560001_rf_2.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Cofe_1560001_rf_3.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Cofe_1560001_rf_3.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Cofe_1560001_rf_4.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Cofe_1560001_rf_4.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Cofe_1560001_rf_5.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Cofe_1560001_rf_5.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\

...

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_1560002_rf_12.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_1560002_rf_12.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_1560002_rf_2.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_1560002_rf_2.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_1560002_rf_3.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_1560002_rf_3.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_1560002_rf_4.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_1560002_rf_4.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 201

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_2750000_rf_5.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_2750000_rf_5.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_2750000_rf_6.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_2750000_rf_6.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_2750000_rf_7.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_2750000_rf_7.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_2750000_rf_8.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_2750000_rf_8.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_4100006_rf_3.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_4100006_rf_3.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_4100006_rf_4.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_4100006_rf_4.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_4100006_rf_5.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_4100006_rf_5.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_4100006_rf_6.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_4100006_rf_6.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_640011_rf_1.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_640011_rf_1.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_640011_rf_11.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_640011_rf_11.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_640011_rf_12.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_640011_rf_12.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe1_640011_rf_2.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe1_640011_rf_2.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\Dist

Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe2_4100007_rf_7.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe2_4100007_rf_7.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe2_4100007_rf_8.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe2_4100007_rf_8.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe2_4100007_rf_9.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe2_4100007_rf_9.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\Rainfed - Copy\Whe2_4100009_rf_5.0.tif" to "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\District\New folder\Whe2_4100009_rf_5.tif"
Copied "D:\PhD_Udel\MICRA_2015\DOwnscaling\Asia\All countries\Maps 2010\